In [10]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [3]:
df = pd.read_excel(r'D:\OneDrive\Personal\OneDrive\Projects\vietnam-districts\vietnam-districts.xlsx',sheet_name='Sheet1')

In [4]:
url_dict = dict(zip(df['Geo_unit'], df['Wiki_URL'])) 

In [5]:
area = []
with requests.Session() as s:
    for district, url in url_dict.items():
        r = s.get(url)
        soup = bs(r.content, 'lxml')
        result = soup.select('table > tbody > tr > th + td:contains(km)')
        if not result:
            # print(district, 'failed')
            l = [district, None]
            area.append(l)
        else:
            # print(district, result[0].get_text(strip=True))
            l = [district, result[0].get_text(strip=True).replace('\xa0', ' ')]
            area.append(l)

In [6]:
area_df  = pd.DataFrame(area, columns=['Geo_unit', 'Area'])

In [7]:
df = df.merge(area_df, how='left',on='Geo_unit')

In [11]:
locator = Nominatim(user_agent="myGeocoder")
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df['location'] = df['Geo_unit_full_name'].apply(geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

df = df.drop(['point'], axis=1)

In [15]:
df.drop(['Wiki_URL','location','altitude'], axis=1).to_csv(r'D:\OneDrive\Personal\OneDrive\Projects\vietnam-districts\vietnam-districts.csv')